Para ejecutar este código se necesitan 2 archivos en el mismo directorio que el código y la carpeta de imagenes de matriculas a detectar.

Los archivos són los siguientes:
- best.pt
- MESPREG.ttf

In [7]:
#Path de la carpeta con imagenes de matriculas
path = 'images'

In [13]:
!pip install -r requirements.txt

     ---------------------------------------- 35.1/35.1 MB 3.2 MB/s eta 0:00:00


ERROR: Could not install packages due to an OSError: [WinError 5] Acceso denegado: 'C:\\Users\\rober\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python39\\site-packages\\~v2\\cv2.cp39-win_amd64.pyd'
Check the permissions.



  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.5.2.52
    Uninstalling opencv-python-4.5.2.52:
      Successfully uninstalled opencv-python-4.5.2.52


In [3]:
import torch
import cv2
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import easyocr
from PIL import Image, ImageDraw, ImageFont

c:\Users\javie\anaconda\envs\opencv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt')

Using cache found in C:\Users\javie/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-10-8 Python-3.9.13 torch-1.12.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [5]:
path_results = "resultados/"
if not os.path.exists(path_results):
   os.makedirs(path_results)

In [13]:
p = 0.03
correct = 0
incorrect = 0
def get_optimal_font_scale(text, width):

    for scale in reversed(range(0, 60, 1)):
        textSize = cv2.getTextSize(text, fontFace=cv2.FONT_HERSHEY_DUPLEX, fontScale=scale/10, thickness=1)
        new_width = textSize[0][0]
        if (new_width <= width):
            return int(scale/10)
    return 1
reader = easyocr.Reader(['es'])
for n, file in enumerate(os.listdir(path)):
    #Read image
    img = cv2.imread(path+"/"+file)

    #Generate crop
    results = model(img)
    coor = results.pandas().xyxy[0]
    coor = coor.astype({'xmin':'int', 'ymin':'int', 'xmax': 'int', 'ymax': 'int'})
    if len(coor) == 0:
        img = Image.fromarray(img)
        I = ImageDraw.Draw(img)
        myFont = ImageFont.truetype('MESPREG.ttf', 200)

        I.text((100, 10), "No se detecta matricula", fill=(242, 190, 0), font=myFont)
        cv2.imwrite(path_results+file, np.array(img))
        continue
    crop = img[coor["ymin"][0]-int(img.shape[1]*p):coor["ymax"][0]+int(img.shape[1]*p), coor["xmin"][0]-int(img.shape[0]*p):coor["xmax"][0]+int(img.shape[0]*p)]

    #Read text
    result = reader.readtext(crop, allowlist="BCDFGHJKLMNPQRSTVWXYZ1234567890")
    matricula = ""
    for detection in result:
        if detection[2] > 0.5:
            matricula = matricula + ' ' + detection[1]
    plate = matricula.split()
    plate.sort(key=len, reverse = True)
    plate = ''.join(plate)[:7].upper()
    plate = plate[0:4].replace("L","4") + plate[4:]
    plate = plate[:4] + plate[4:7].replace("4","L")

    #Generate image with plate
    img = Image.fromarray(img)
    I = ImageDraw.Draw(img)
    myFont = ImageFont.truetype('MESPREG.ttf', 300)

    I.text((100, 10), plate, fill=(242, 190, 0), font=myFont)
    cv2.imwrite(path_results+file, np.array(img))

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
